In [239]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
from dateutil.relativedelta import relativedelta

# Reading Data

In [240]:
campaigns = pd.read_csv('./data/campaigns.csv')

donors = pd.read_csv('./data/donors.csv')

gifts = pd.read_csv('./data/gifts.csv')

campaigns_6169 = pd.read_csv('./data/selection campaign 6169.csv')

# GIFTS

### Checking data in gifts

In [241]:

print(gifts.isna().sum().sum())
for column in gifts.columns[0:]:
    print(column + " " + str(gifts[column].isna().sum()))

gifts.info()

15637
Unnamed: 0 0
donorID 0
campaignID 15637
amount 0
date 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216594 entries, 0 to 216593
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  216594 non-null  int64  
 1   donorID     216594 non-null  int64  
 2   campaignID  200957 non-null  float64
 3   amount      216594 non-null  object 
 4   date        216594 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 8.3+ MB


### Cleaning data in gifts

In [242]:
gifts['campaignID'] = gifts['campaignID'].fillna(0.0)
gifts['campaignID'] = gifts['campaignID'].astype(int)
gifts['amount'] = gifts['amount'].str.replace(",", ".").astype(float)
gifts['date'] = pd.to_datetime(gifts['date'])
gifts.rename(columns={'Unnamed: 0' : 'giftID'}, inplace=True)
gifts.head()

,giftID,donorID,campaignID,amount,date
0,0,100001,0,12.39,2004-03-23
1,1,100001,1577,12.39,2007-10-11
2,2,100001,1634,6.69,2007-12-28
3,3,100001,1778,24.79,2008-06-14
4,4,100001,1808,7.44,2008-07-09


# DONORS

### Checking data in donors

In [243]:
print(donors.isna().sum().sum())
for column in donors.columns[0:]:
    print(column + " " + str(donors[column].isna().sum()))
donors.head()

34
Unnamed: 0 0
donorID 0
zipcode 2
province 0
region 0
gender 32
language 0
dateOfBirth 0


,Unnamed: 0,donorID,zipcode,province,region,gender,language,dateOfBirth
0,0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967
1,1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952
2,2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986
3,3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952
4,4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986


### Cleaning data in donors

In [244]:
donors_zipcode_na = donors.loc[donors['zipcode'].isna(), :]
donors_zipcode_na.head()
donors['zipcode'] = donors['zipcode'].fillna(0)

In [245]:
donors_gender_na = donors.loc[donors['gender'].isna(), :]
donors_gender_na.head()
donors['gender'] = donors['gender'].fillna('Not Known')

In [246]:
donors.drop(['Unnamed: 0'] , axis = 1, inplace=True)
donors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44691 entries, 0 to 44690
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   donorID      44691 non-null  int64  
 1   zipcode      44691 non-null  float64
 2   province     44691 non-null  object 
 3   region       44691 non-null  object 
 4   gender       44691 non-null  object 
 5   language     44691 non-null  object 
 6   dateOfBirth  44691 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 2.4+ MB


# CAMPAIGNS

### Checking data in campaigns

In [247]:
print(campaigns.isna().sum().sum())
for column in campaigns.columns[0:]:
    print(column + " " + str(campaigns[column].isna().sum()))

campaigns.info()

0
campaignID;date;lettersSent;CostUnit 0
<class 'pandas.core.frame.DataFrame'>
Index: 367 entries, 153;13/12/2004;6 873;0 to 7536;15/12/2019;37 187;0
Data columns (total 1 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   campaignID;date;lettersSent;CostUnit  367 non-null    object
dtypes: object(1)
memory usage: 5.7+ KB


In [248]:
campaigns.head()

,campaignID;date;lettersSent;CostUnit
153;13/12/2004;6 873;0,17 €
154;01/01/2005;7 656;0,30 €
180;18/01/2005;9 933;0,20 €
433;29/01/2005;7 448;0,59 €
476;12/02/2005;6 605;0,51 €


### Cleaning data in campaigns

In [249]:
campaigns = pd.read_csv('./data/campaigns.csv')
campaigns.rename(columns={campaigns.columns[0]: "Cost" }, inplace = True)
campaigns.index.names = ['campaignID;date;lettersSent;CostUnit']
campaigns_reset = campaigns.reset_index()
campaigns_split = campaigns_reset['campaignID;date;lettersSent;CostUnit'].str.split(';', expand=True)
campaigns_split['cost'] = campaigns_reset['Cost']
campaigns_split.rename(columns={0: "campaignID" }, inplace = True)
campaigns_split.rename(columns={1: "date" }, inplace = True)
campaigns_split.rename(columns={2: "lettersSent" }, inplace = True)
campaigns_split.drop([3] , axis = 1, inplace=True)
campaigns_split.head()

,campaignID,date,lettersSent,cost
0,153,13/12/2004,6 873,17 €
1,154,01/01/2005,7 656,30 €
2,180,18/01/2005,9 933,20 €
3,433,29/01/2005,7 448,59 €
4,476,12/02/2005,6 605,51 €


In [250]:
campaigns_split['campaignID'] = campaigns_split['campaignID'].astype(int)
campaigns_split['date'] = pd.to_datetime(campaigns_split['date'])
campaigns_split['lettersSent'] = campaigns_split['lettersSent'].str.replace("\u202f", "").astype(int)
campaigns_split['cost'] = campaigns_split['cost'].str.replace("\xa0€", "").astype(int)
campaigns = campaigns_split
campaigns.head()


,campaignID,date,lettersSent,cost
0,153,2004-12-13,6873,17
1,154,2005-01-01,7656,30
2,180,2005-01-18,9933,20
3,433,2005-01-29,7448,59
4,476,2005-12-02,6605,51


In [251]:
campaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   campaignID   367 non-null    int32         
 1   date         367 non-null    datetime64[ns]
 2   lettersSent  367 non-null    int32         
 3   cost         367 non-null    int32         
dtypes: datetime64[ns](1), int32(3)
memory usage: 7.3 KB


### Creating Features

In [252]:
#Calculating summary for donors till Aug 4th 2018
gifts_donor_campaignID = gifts[gifts['date'] < datetime.datetime(2018, 8, 4)].groupby('donorID', as_index=False)['campaignID'].agg({'number_of_campaigns':'count'})
gifts_donor_total_amount = gifts[gifts['date'] < datetime.datetime(2018, 8, 4)].groupby('donorID', as_index=False)['amount'].agg({'total_amount':'sum'})
gifts_donor_last_date = gifts[gifts['date'] < datetime.datetime(2018, 8, 4)].groupby('donorID', as_index=False)['date'].agg({'last_date':'max'})

donors = donors.merge(gifts_donor_campaignID, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_total_amount, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors = donors.merge(gifts_donor_last_date, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors['days_since_contributed'] = datetime.datetime(2018, 8, 4) - donors['last_date'] 
donors.head() 

,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,days_since_contributed
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,12.0,178.49,2011-12-30,2409 days
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,1302 days
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,5.0,18.00,2018-07-08,27 days
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,26.0,716.22,2011-06-03,2619 days
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,4.0,14.96,2018-06-19,46 days


In [253]:
#Calculating summary for donors till Aug 4th 2018
gifts_donor_summary = gifts[gifts['date'] < datetime.datetime(2018, 8, 4)].groupby(['donorID'], as_index=False)['amount'].agg(['mean', 'min', 'max'])
donors = donors.merge(gifts_donor_summary, left_on = 'donorID', right_on = 'donorID', how = 'left')
donors.head()

,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,days_since_contributed,mean,min,max
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,12.0,178.49,2011-12-30,2409 days,14.874167,6.69,30.00
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,1302 days,20.000000,20.00,20.00
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,5.0,18.00,2018-07-08,27 days,3.600000,3.00,5.00
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,26.0,716.22,2011-06-03,2619 days,27.546923,24.79,49.58
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,4.0,14.96,2018-06-19,46 days,3.740000,2.48,5.00


In [254]:
#Calculating age for donors till Aug 4th 2018
donors.loc[:,'age'] = donors['dateOfBirth'].apply(lambda x : datetime.datetime(2018, 8, 4).year - pd.to_datetime(x).year)
donors.loc[:,'age-group'] = donors['age'] // 10 * 10
donors.head()

,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,days_since_contributed,mean,min,max,age,age-group
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,12.0,178.49,2011-12-30,2409 days,14.874167,6.69,30.00,51,50
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,1302 days,20.000000,20.00,20.00,66,60
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,5.0,18.00,2018-07-08,27 days,3.600000,3.00,5.00,32,30
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,26.0,716.22,2011-06-03,2619 days,27.546923,24.79,49.58,66,60
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,4.0,14.96,2018-06-19,46 days,3.740000,2.48,5.00,32,30


In [255]:
gifts.head()

,giftID,donorID,campaignID,amount,date
0,0,100001,0,12.39,2004-03-23
1,1,100001,1577,12.39,2007-10-11
2,2,100001,1634,6.69,2007-12-28
3,3,100001,1778,24.79,2008-06-14
4,4,100001,1808,7.44,2008-07-09


In [256]:
campaigns_6169.nunique()

donorID    34888
dtype: int64

In [257]:
selected_donors = donors.merge(campaigns_6169, left_on='donorID', right_on='donorID', how = 'inner')
selected_donors.head()
selected_donors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34874 entries, 0 to 34873
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   donorID                 34874 non-null  int64          
 1   zipcode                 34874 non-null  float64        
 2   province                34874 non-null  object         
 3   region                  34874 non-null  object         
 4   gender                  34874 non-null  object         
 5   language                34874 non-null  object         
 6   dateOfBirth             34874 non-null  object         
 7   number_of_campaigns     34869 non-null  float64        
 8   total_amount            34869 non-null  float64        
 9   last_date               34869 non-null  datetime64[ns] 
 10  days_since_contributed  34869 non-null  timedelta64[ns]
 11  mean                    34869 non-null  float64        
 12  min                     34869 no

In [258]:
donors 

,donorID,zipcode,province,region,gender,language,dateOfBirth,number_of_campaigns,total_amount,last_date,days_since_contributed,mean,min,max,age,age-group
0,100001,1861.0,Flemish Brabant,Flanders,F,NL,12/24/1967,12.0,178.49,2011-12-30,2409 days,14.874167,6.69,30.00,51,50
1,100002,2260.0,Antwerp,Flanders,M,NL,01/14/1952,1.0,20.00,2015-01-10,1302 days,20.000000,20.00,20.00,66,60
2,100003,1780.0,Flemish Brabant,Flanders,M,NL,12/23/1986,5.0,18.00,2018-07-08,27 days,3.600000,3.00,5.00,32,30
3,100004,1020.0,Brussels,Brussels,F,FR,03/29/1952,26.0,716.22,2011-06-03,2619 days,27.546923,24.79,49.58,66,60
4,100005,1020.0,Brussels,Brussels,F,FR,06/25/1986,4.0,14.96,2018-06-19,46 days,3.740000,2.48,5.00,32,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44686,144687,4900.0,Liege,Wallonia,M,FR,09/12/1970,1.0,10.00,2015-11-06,1002 days,10.000000,10.00,10.00,48,40
44687,144688,3600.0,Limburg,Flanders,M,NL,11/28/1972,7.0,48.56,2017-07-27,373 days,6.937143,2.48,15.00,46,40
44688,144689,3061.0,Flemish Brabant,Flanders,M,NL,04/21/1967,2.0,124.79,2018-06-25,40 days,62.395000,24.79,100.00,51,50
44689,144690,1950.0,Flemish Brabant,Flanders,M,FR,05/07/1976,1.0,40.00,2014-12-30,1313 days,40.000000,40.00,40.00,42,40


In [259]:
# Checking for mutiple donations for campaign 6169
gifts1 = gifts[gifts['campaignID'] == 6169]
agg_gifts = gifts1.groupby(['donorID'])['date'].agg(['count']).sort_values(by = 'count', ascending =False)


In [260]:
selected_donors = selected_donors.merge(gifts1, left_on='donorID', right_on='donorID', how = 'left')
# If Contribution made after May 18 

In [261]:
selected_donors['amount'].fillna(0)
#selected_donors.loc[(selected_donors['amount'] > 30) and (selected_donors['date'] > datetime.datetime(2018, 9, 4)) and (selected_donors['date'] < datetime.datetime(2019, 6, 18)), 'has_contributed'] = 1

selected_donors['has_contributed'] = np.where((selected_donors['amount'] > 30) & (selected_donors['date'] > datetime.datetime(2018, 9, 4)) & (selected_donors['date'] < datetime.datetime(2019, 6, 18)),1, 0)